In [2]:
from math import log
#计算信息熵,，熵越高，混合的信息越多
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featureVec in dataSet:
        currentLabel = featureVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] +=1
    shannonEnt = 0.0
    for key in labelCounts:
        #概率p(X)
        prob = float(labelCounts[key] / numEntries)
        shannonEnt -= prob * log(prob, 2)
    return shannonEnt

In [3]:
#划分数据集
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featureVec in dataSet:
        #print(featureVec)
        if featureVec[axis] == value:
            #相当于reducedFeatVec = []
            reducedFeatVec = featureVec[: axis]
            reducedFeatVec.extend(featureVec[axis+1:])
            retDataSet.append(featureVec[axis+1:])
    return retDataSet

In [4]:
def creatDataSet():
    dataSet = [
        [1, 1, 'yes'],
        [1, 0, 'no'],
        [0, 1, 'no'],
        [0, 1, 'no'],
    ]
    labels = ['no surrfacing', 'flippers']
    return dataSet, labels

In [5]:
data, labels= creatDataSet()
splitDataSet(data, 0, 1)

[[1, 'yes'], [0, 'no']]

In [29]:
def chooseBestFeatureSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet)
    #print("base：%f" % baseEntropy)
    bestInfoGain = 0.0 
    bestFeature = -1
    for i in range(numFeatures):
        featureList = [example[i] for example in dataSet]
        #print(featureList)
        uniqueVals = set(featureList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
            #print("newEntropy: %s" % newEntropy)
        infoGain = baseEntropy - newEntropy
        #print("infoGain: %s, bestInfoGain: %s" % (infoGain, bestInfoGain))
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

In [7]:
chooseBestFeatureSplit(data)

base：0.811278
[1, 1, 0, 0]
newEntropy: 0.0
newEntropy: 0.5
infoGain: 0.31127812445913283, bestInfoGain: 0.0
[1, 0, 1, 1]
newEntropy: 0.0
newEntropy: 0.6887218755408672
infoGain: 0.12255624891826566, bestInfoGain: 0.31127812445913283


0

In [8]:
import operator
def majortyCount(classList):
    classCount ={}
    for vote in classList:
        if vote in classList.keys():
            classCount[vote] = 0
        classCount += 1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [39]:
def createTree(dataSet, labels):
    classList = [example[-1] for example in dataSet]
    print("classList %s" % classList)
    print("dataSet %s" % dataSet)
    #列表第一个类别的个数与列表元素个数是否一致， 即只剩下一个类别时结束递归
    if classList.count(classList[0])  == len(classList):
        return classList[0]
    if len(dataSet[0]) == 1:
        return majortyCount(classList)
    bestFeat = chooseBestFeatureSplit(dataSet)
    print(bestFeat)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in  dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value), subLabels)
    return myTree

In [59]:
dataSet, labels = creatDataSet()
myTree = createTree(dataSet, labels)
myTree

classList ['yes', 'no', 'no', 'no']
dataSet [[1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
0
classList ['no', 'no']
dataSet [[1, 'no'], [1, 'no']]
classList ['yes', 'no']
dataSet [[1, 'yes'], [0, 'no']]
0
classList ['no']
dataSet [['no']]
classList ['yes']
dataSet [['yes']]


{'no surrfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

In [68]:
def classify(inputTree, featLabels, testVec):
    firstStr = list(inputTree.keys())[0]
    secondDict = inputTree[firstStr]
    #print(featLabels)
    featIndex = featLabels.index(firstStr)
    for key in secondDict.keys():
        if testVec[featIndex] == key:
            if  type(secondDict[key]).__name__ == 'dict':
                classLabel = classify(secondDict[key], featLabels, testVec)
            else:
                classLabel = secondDict[key]
    return classLabel

In [67]:
dataSet, labels = creatDataSet()
classify(myTree, labels, [1,0])

dict_keys(['no surrfacing'])
dict_keys(['flippers'])


'no'

In [69]:
#保存决策树(字典对象保存)
import pickle
def storeTree(inputTree, filename):
    fw = open(filename,'w')
    pickle.dump(inputTree, fw)
    fw.close()

def getTree(filename):
    fr = open(filename,'r')
    return pickle.load(fr)
    